In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from deepforest import CascadeForestClassifier

In [2]:
train = pd.read_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/train.csv')
test = pd.read_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/test.csv')

target = pd.get_dummies(train['target'])
y = train['target']

X = train.iloc[:,1:-1]
test = test.iloc[:,1:]

In [3]:
model = CascadeForestClassifier(predictor="xgboost", predictor_kwargs={'eval_metric': "logloss"})
model.fit(np.array(X), np.array(train['target']))

[2021-06-08 16:22:32.802] Start to fit the model:
[2021-06-08 16:22:32.802] Fitting cascade layer = 0 
[2021-06-08 16:25:00.204] layer = 0  | Val Acc = 35.319 % | Elapsed = 147.402 s
[2021-06-08 16:25:01.270] Fitting cascade layer = 1 
[2021-06-08 16:28:28.074] layer = 1  | Val Acc = 35.100 % | Elapsed = 206.803 s
[2021-06-08 16:28:28.074] Early stopping counter: 1 out of 2
[2021-06-08 16:28:29.081] Fitting cascade layer = 2 
[2021-06-08 16:31:56.036] layer = 2  | Val Acc = 35.040 % | Elapsed = 206.955 s
[2021-06-08 16:31:56.036] Early stopping counter: 2 out of 2
[2021-06-08 16:31:56.036] Handling early stopping
[2021-06-08 16:31:56.244] The optimal number of layers: 1


In [4]:
preds = model.predict_proba(np.array(test))

test = pd.read_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/test.csv')
submission = pd.DataFrame(preds, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])
submission['id'] = test['id']
submission = submission[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']]

[2021-06-08 16:31:56.543] Start to evalute the model:
[2021-06-08 16:31:56.702] Evaluating cascade layer = 0 


In [5]:
submission.to_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/submission_2.csv', index=False)